In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, Normalizer, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [2]:
df = pd.read_csv('learningSet.csv', dtype = {'NOEXCH': str})
df

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,ASE,1,AK,99504,,,0,0,,...,0.0,0,L,1,G,X,X,X,12.0,C
95408,9601,DCD,1,TX,77379,,,5001,0,,...,0.0,1,L,1,F,X,X,X,2.0,A
95409,9501,MBC,1,MI,48910,,,3801,0,,...,0.0,1,L,3,E,X,X,X,34.0,B
95410,8601,PRV,0,CA,91320,,,4005,0,X,...,18.0,1,L,4,F,X,X,X,11.0,A


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1. Check for null values in all the columns

In [4]:
nulls = pd.DataFrame(df.isna().sum(), columns=['nan_count'])
nulls = nulls[nulls['nan_count'] > 0]
nulls.sort_values('nan_count', ascending = False)

,nan_count
RDATE_5,95403
RAMNT_5,95403
RAMNT_3,95170
RDATE_3,95170
RDATE_4,95131
RAMNT_4,95131
RDATE_6,94636
RAMNT_6,94636
RDATE_15,88150
RAMNT_15,88150


## 2. Create drop list and exclude OSOURCE and ZIP

In [5]:
drop_list = []
drop_list.append('OSOURCE')
drop_list.append('ZIP')

In [6]:
drop_list

['OSOURCE', 'ZIP']

## 3. Identify columns that over 85% missing values

In [7]:
null_cols = pd.DataFrame(df.isna().sum()*100/len(df), columns=['nan_%'])
null_cols = null_cols[null_cols['nan_%'] > 85]
null_cols.sort_values('nan_%', ascending = False)

,nan_%
RDATE_5,99.990567
RAMNT_5,99.990567
RDATE_3,99.746363
RAMNT_3,99.746363
RDATE_4,99.705488
RAMNT_4,99.705488
RDATE_6,99.186685
RAMNT_6,99.186685
RAMNT_15,92.388798
RDATE_15,92.388798


## 4. Remove those columns from the dataframe

In [8]:
drop_list.extend(list(null_cols.index))

In [9]:
drop_list

['OSOURCE',
 'ZIP',
 'NUMCHLD',
 'RDATE_3',
 'RDATE_4',
 'RDATE_5',
 'RDATE_6',
 'RDATE_7',
 'RDATE_10',
 'RDATE_13',
 'RDATE_15',
 'RDATE_17',
 'RDATE_20',
 'RDATE_21',
 'RDATE_23',
 'RAMNT_3',
 'RAMNT_4',
 'RAMNT_5',
 'RAMNT_6',
 'RAMNT_7',
 'RAMNT_10',
 'RAMNT_13',
 'RAMNT_15',
 'RAMNT_17',
 'RAMNT_20',
 'RAMNT_21',
 'RAMNT_23']

In [10]:
df = df.drop(columns=drop_list)

## 5. Reduce the number of categories in GENDER (M, F, other)

In [11]:
print(df['GENDER'].value_counts())
df['GENDER'] = df['GENDER'].fillna('F')
df['GENDER'].value_counts()

F    51277
M    39094
      2957
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64


F    51277
M    39094
      2957
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64

In [12]:
#seems empty category is not Nan, but empty spaces
df['GENDER'].isna().sum()

0

In [13]:
df.loc[(df["GENDER"] == "U") | (df["GENDER"] == "J") | (df["GENDER"] == "C") | (df["GENDER"] == "A") | (df["GENDER"] == " "), "GENDER"] = 'other'

In [14]:
df['GENDER'].value_counts()

F        51277
M        39094
other     5041
Name: GENDER, dtype: int64